In [2]:
%pip install gdown

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
import gdown
from io import StringIO

# Google Drive file ID
file_id = "1UZ_Qth0sJ_x640QX7mFjW2HVb91ua00l"

# Download file as text in memory
url = f"https://drive.google.com/uc?id={file_id}"
downloaded_file = gdown.download(url, quiet=False, fuzzy=True)

# Read directly into pandas
df = pd.read_csv(downloaded_file)

print(df.head())
print("SHOW ALL COLUMNS")
for col in df.columns:
    print(col)







Note: you may need to restart the kernel to use updated packages.


Downloading...
From (original): https://drive.google.com/uc?id=1UZ_Qth0sJ_x640QX7mFjW2HVb91ua00l
From (redirected): https://drive.google.com/uc?id=1UZ_Qth0sJ_x640QX7mFjW2HVb91ua00l&confirm=t&uuid=003c973c-6aa7-4d73-ad68-8fd4b8148be1
To: d:\Users\daseb\OneDrive\Desktop\Project\ACADS\Courses\CS12\L-Machine Learning\survey_results_public.csv
100%|██████████| 160M/160M [00:26<00:00, 5.94MB/s] 


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

## Total row is 65437



In [ ]:
df_col = df[['Age','Employment','YearsCode','YearsCodePro','DevType','EdLevel','ConvertedCompYearly']].copy()


df_devtype = pd.DataFrame(df_col["DevType"].unique())
df_yrscode = pd.DataFrame(df_col["YearsCode"].unique())
df_yrscodepro = pd.DataFrame(df_col["YearsCodePro"].unique())
df_age= pd.DataFrame(df_col["Age"].unique())
df_edlvl= pd.DataFrame(df_col["EdLevel"].unique())

## checking  unique data in a column

display(df_edlvl)

,0
0,Primary/elementary school
1,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
2,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
3,Some college/university study without earning ...
4,"Secondary school (e.g. American high school, G..."
5,"Professional degree (JD, MD, Ph.D, Ed.D, etc.)"
6,"Associate degree (A.A., A.S., etc.)"
7,Something else
8,NaN



- **Goal:** Predict salary based on experience, education, and developer type.  
- **Missing values:**  
  - `YearsCode` / `YearsCodePro` → filled with 0 (no experience).  
  - `DevType` / `Employment` → filled with "Unknown" (type not specified).  
- **Age:** Converted ranges like "25-34" to numeric (29.5) for the model.  
- **Education (EdLevel):** Mapped categories to numbers (0 = primary, 1 = bachelor’s, 2 = master’s).  
- **One-Hot Encoding:** Used for `DevType` and `Employment` to treat each category independently.  
- **Reasoning:** These steps ensure the model can handle missing data, numeric features, and categorical variables correctly while keeping it simple and interpretable.


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 1. Select relevant columns and make a copy
df_col = df[['Age','Employment','YearsCode','YearsCodePro','DevType','EdLevel','ConvertedCompYearly']].copy()
df_col = df_col.reset_index(drop=True)

# 2. Drop rows without target
df_col = df_col.dropna(subset=["ConvertedCompYearly"])

# 3. Fill missing values
df_col["DevType"] = df_col["DevType"].fillna("Unknown")
df_col["YearsCodePro"] = df_col["YearsCodePro"].fillna(0)
df_col["YearsCode"] = df_col["YearsCode"].fillna(0)
df_col["Employment"] = df_col["Employment"].fillna("Unknown")

def convert_experience(x):
    if pd.isna(x):
        return 0
    if isinstance(x, str):
        x = x.strip()
        if x == "Less than 1 year":
            return 0.5
        elif x == "More than 50 years":
            return 50
        else:
            try:
                return float(x)
            except:
                return 0
    return x

df_col['YearsCode'] = df_col['YearsCode'].apply(convert_experience)
df_col['YearsCodePro'] = df_col['YearsCodePro'].apply(convert_experience)

# 4. Map Age
age_mapping = {
    "18-24 years old": 21,
    "25-34 years old": 29.5,
    "35-44 years old": 39.5,
    "45-54 years old": 49.5,
    "55-64 years old": 59.5,
    "65 years or older": 65.5,
    "Under 18 years old": 18.5,
    "Prefer not to say": 0
}
df_col["Age"] = df_col["Age"].map(age_mapping).fillna(0)

# 5. Map EdLevel (ordinal encoding)
edlevel_mapping = {
    "Primary/elementary school": 0.0,
    "Secondary school (e.g. American high school, G...)": 1.0,
    "Some college/university study without earning ...": 2.0,
    "Associate degree (A.A., A.S., etc.)": 3.0,
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": 4.0,
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": 5.0,
    "Professional degree (JD, MD, Ph.D, Ed.D, etc.)": 6.0,
    "Something else": 7.0,
    # handle missing values
    np.nan: -1.0
}
df_col["EdLevel"] = df_col["EdLevel"].map(edlevel_mapping).fillna(-1.0)

# 6. One-Hot Encode Employment
ohe_emp = OneHotEncoder(sparse_output=False, drop=None)
employment_encoded = ohe_emp.fit_transform(df_col[['Employment']])
employment_encoded_df = pd.DataFrame(employment_encoded, columns=ohe_emp.get_feature_names_out(['Employment']), index=df_col.index)
df_col = pd.concat([df_col, employment_encoded_df], axis=1)
df_col = df_col.drop('Employment', axis=1)

# 7. One-Hot Encode DevType
ohe_dev = OneHotEncoder(sparse_output=False, drop=None)
devtype_encoded = ohe_dev.fit_transform(df_col[['DevType']])
devtype_encoded_df = pd.DataFrame(devtype_encoded, columns=ohe_dev.get_feature_names_out(['DevType']), index=df_col.index)
df_col = pd.concat([df_col, devtype_encoded_df], axis=1)
df_col = df_col.drop('DevType', axis=1)

# 8. Final dataframe ready
display(df_col)
print(df_col.count())


,Age,YearsCode,YearsCodePro,EdLevel,ConvertedCompYearly,"Employment_Employed, full-time","Employment_Employed, full-time;Employed, part-time","Employment_Employed, full-time;Independent contractor, freelancer, or self-employed","Employment_Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time","Employment_Employed, full-time;Independent contractor, freelancer, or self-employed;Not employed, and not looking for work",...,DevType_Other (please specify):,DevType_Product manager,DevType_Project manager,DevType_Research & Development role,DevType_Scientist,DevType_Security professional,"DevType_Senior Executive (C-Suite, VP, etc.)",DevType_Student,DevType_System administrator,DevType_Unknown
72,21.00,3.00,1.00,-1.00,"7,322.00",0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
374,29.50,12.00,6.00,6.00,"30,074.00",1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
379,39.50,15.00,6.00,5.00,"91,295.00",1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
385,39.50,27.00,17.00,5.00,"53,703.00",0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
389,29.50,7.00,7.00,-1.00,"110,000.00",0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41180,29.50,6.00,3.00,5.00,"44,640.00",1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
41184,49.50,36.00,30.00,5.00,"170,000.00",1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
41185,39.50,20.00,15.00,4.00,"116,844.00",1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
41186,21.00,5.00,1.00,-1.00,"12,000.00",0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Age                                             23435
YearsCode                                       23435
YearsCodePro                                    23435
EdLevel                                         23435
ConvertedCompYearly                             23435
                                                ...  
DevType_Security professional                   23435
DevType_Senior Executive (C-Suite, VP, etc.)    23435
DevType_Student                                 23435
DevType_System administrator                    23435
DevType_Unknown                                 23435
Length: 95, dtype: int64


In [ ]:

df_col.isnull().sum()


Age                                             0
YearsCode                                       0
YearsCodePro                                    0
EdLevel                                         0
ConvertedCompYearly                             0
                                               ..
DevType_Security professional                   0
DevType_Senior Executive (C-Suite, VP, etc.)    0
DevType_Student                                 0
DevType_System administrator                    0
DevType_Unknown                                 0
Length: 95, dtype: int64

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df_col.drop('ConvertedCompYearly', axis=1)
y = df_col['ConvertedCompYearly']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_cols = ['Age', 'YearsCode', 'YearsCodePro', 'EdLevel']

scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 1. Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# 2. Make predictions
y_pred = model.predict(X_test)

# 3. Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:,.2f}")
print(f"R2 Score: {r2:.4f}")
print("===================================================\n")

# Example: pick one row from X_test to predict
sample = X_test.iloc[400]  # first row of test set
print("Sample features:\n", sample)

# Reshape to 2D array for prediction
sample_array = sample.values.reshape(1, -1)

# Predict using the trained model
print("===================================================\n")
predicted_salary = model.predict(sample_array)
print(f"Predicted Salary: ${predicted_salary[0]:,.2f}")

# Compare with actual
actual_salary = y_test.iloc[0]
print(f"Actual Salary: ${actual_salary:,.2f}")


X_train shape: (18748, 94)
X_test shape: (4687, 94)
y_train shape: (18748,)
y_test shape: (4687,)
RMSE: 93,724.93
R2 Score: 0.0630

Sample features:
 Age                                            -0.47
YearsCode                                      -0.31
YearsCodePro                                   -0.48
EdLevel                                         0.22
Employment_Employed, full-time                  1.00
                                                ... 
DevType_Security professional                   0.00
DevType_Senior Executive (C-Suite, VP, etc.)    0.00
DevType_Student                                 0.00
DevType_System administrator                    0.00
DevType_Unknown                                 0.00
Name: 7949, Length: 94, dtype: float64

Predicted Salary: $76,269.43
Actual Salary: $32,640.00


c:\Users\daseb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


# Salary Prediction Model – Full Reflection

## 1. Objective
- The goal of this project is to **predict a developer’s salary** based on a subset of features:  
  - Age  
  - Years of coding experience (YearsCode, YearsCodePro)  
  - Education level (EdLevel)  
  - Developer type (DevType)  
- *Source:* Data is derived from the [Stack Overflow Developer Survey](https://insights.stackoverflow.com/survey), using only a few columns for simplicity and focus.  
- Reflection: Using a subset of features makes the model more interpretable but may leave out other important factors like location, company size, or industry trends.

---

## 2. Data Preprocessing

### a) Selecting and Copying Columns
- I selected only the columns relevant to the prediction task and made a copy to avoid altering the original dataset.  
- Reflection: This ensures that preprocessing steps do not accidentally modify the raw data and keeps the workflow organized.

### b) Dropping Irrelevant Columns
- Columns that **do not directly affect the target salary** (e.g., extra metadata) were dropped.  
- Reflection: Removing irrelevant columns reduces noise and improves model focus on meaningful predictors.

### c) Handling Missing Values
- Missing numeric values (YearsCode, YearsCodePro) were treated as 0, assuming no experience.  
- Missing categorical values (DevType, Employment) were filled with `"Unknown"`.  
- Reflection: Handling nulls in a thoughtful way is critical. Treating missing experience as zero is a simple, human-understandable assumption that allows the model to learn patterns without biasing toward missing data.

### d) Mapping and Encoding
- **Age** and **EdLevel** were mapped to numeric values to represent ranges and education hierarchy.  
- **Categorical variables** (DevType, Employment) were one-hot encoded so the model can process them numerically.  
- Reflection: This step converts human-readable categories into a format the machine learning model can understand. It highlights the challenge of translating real-world concepts into numbers without losing meaning.

---

## 3. Data Splitting
- Split the data into **training** (80%) and **testing** (20%) sets.  
- Reflection: This is essential for evaluating the model’s ability to generalize to new, unseen data. It simulates real-life scenarios where the model predicts salaries for developers not included in the training dataset.

---

## 4. Feature Scaling
- Numeric columns (Age, YearsCode, YearsCodePro, EdLevel) were standardized using `StandardScaler`.  
- Reflection: Scaling prevents features with large ranges from dominating the model’s learning process. It also improves the convergence and stability of models like Linear Regression.

---

## 5. Model Training
- Trained a **Linear Regression** model using the training data.  
- Reflection: Linear Regression is simple, interpretable, and suitable for predicting a continuous target like salary. However, it assumes linear relationships between features and salary, which may not fully capture real-world complexities.

---

## 6. Prediction
- The model predicted salaries on the test set.  
- Reflection: Comparing predictions with actual salaries demonstrates how well the model generalizes and highlights potential gaps due to missing features or non-linear patterns.

---

## 7. Model Evaluation
- Metrics used:  
  - **RMSE (Root Mean Squared Error):** measures average prediction error.  
  - **R² Score:** measures how much variance in salary is explained by the model.  
- Reflection: These metrics quantify model performance. Even with carefully preprocessed data, using only a few columns limits the predictive power, emphasizing the importance of feature selection in real-world modeling.

---

## 8. Testing on a Single Sample
- Picked a sample row from the test set and predicted the salary, then compared it with the actual salary.  
- Reflection: This step makes the model tangible and interpretable. It shows that individual salaries can vary even with similar experience, highlighting the challenge of predicting human-centered outcomes.

---

## 9. Overall Reflection
- **Preprocessing decisions** (dropping irrelevant columns, filling nulls, mapping, encoding, scaling) significantly affect predictions.  
- Dropping columns like `ComputedCompYearly` (or others that don’t affect the target) reduced noise and improved model focus.  
- **Feature selection** simplifies the model but limits accuracy. Adding variables like location, company, and years since last promotion could improve performance.  
- **Linear Regression** is interpretable but may not capture complex patterns. Other models (Random Forest, Gradient Boosting) could be explored for more nuanced predictions.  
- Predicting salary illustrates the **challenges of real-world data modeling**, including human variability, missing information, and the balance between simplicity and accuracy.

*Dataset source:* Stack Overflow Developer Survey – subset of columns (Age, YearsCode, YearsCodePro, EdLevel, DevType, Employment, ConvertedCompYearly)
